# How does SHA-256 work?
### Simple hasing algorithm 256

In [48]:
from hashlib import sha256

m = sha256()
print(f"Hash type:\t{m.name}")
print(f"Void digest:\t{m.hexdigest()}")
print(f"Digest size:\t{m.digest_size}")

string = "Hello World!"
m.update(string.encode())
print(f"\nString:\t{string}")
print(f"Digest:\t{m.hexdigest()}")


Hash type:	sha256
Void digest:	e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
Digest size:	32

String:	Hello World!
Digest:	7f83b1657ff1fc53b92dc18148a1d65dfc2d4b1fa3d677284addd200126d9069


## Atomic bit level operations

1. SHR
2. ROTR
3. XOR
4. SUM

In [49]:
from bitarray import bitarray
from time import sleep

### 1. SHR
Shift right

In [50]:
def shr(bits, shifts):
    for _ in range(shifts):
        bits.pop()
        bits.insert(0, 0)
    return bits

a = bitarray(endian='big')
a.frombytes(b'\xff\x00\xff\x00')

print("{:>8} {:>32}".format("A:", a.to01()))
for i in range(32):
    print("\r{:>8} {:>32}".format("SHR "+str(i+1)+":", shr(a,1).to01()), end="")
    sleep(0.2)
del a, i

      A: 11111111000000001111111100000000
 SHR 32: 00000000000000000000000000000000

### 2. ROTR
Rotation right

In [51]:
def rotr(bits, shifts):
    for _ in range(shifts):
        bits.insert(0,bits.pop())
    return bits

a = bitarray(endian='big')
a.frombytes(b'\xff\x00\xff\x00')

print("{:>8} {:>32}".format("A:", a.to01()))
for i in range(32):
    print("\r{:>8} {:>32}".format("ROTR "+str(i+1)+":", rotr(a,1).to01()), end="")
    sleep(0.2)
del a, i

      A: 11111111000000001111111100000000
ROTR 32: 11111111000000001111111100000000

### 3. XOR
Exclusive OR



| A | B | XOR |
|---|---|-----|
| 0 | 0 | 0   |
| 0 | 1 | 1   |
| 1 | 0 | 1   |
| 1 | 1 | 0   |   



In [52]:
def xor(a, b):
    return a^b

a = bitarray(endian='big')
b = bitarray(endian='big')
a.frombytes(b'\xff\x00\xff\x00')
b.frombytes(b'\x00\xff\xff\x00')

print("{:>8} {:>32}".format("A:", a.to01()))
print("{:>8} {:>32} XOR".format("B:", b.to01()))
print("{:>8} {:>32}".format("", "-"*32))
for i in range(32):
    print("\r{:>8} {:>32}".format("", xor(a,b).to01()[32-1-i:]), end="")
    sleep(0.2)
del a, b, i


      A: 11111111000000001111111100000000
      B: 00000000111111111111111100000000 XOR
         --------------------------------
         11111111111111110000000000000000

### 4. SUM
Binary addition <br>
Mod 32, ignoring the carry out that will overflow the 32 bit.

In [74]:
def add(a, b):
    result = bitarray(endian='big')
    carry = 0
    for i in range(32-1, -1, -1):
        result.insert(0,(a[i]^b[i]^carry))
        carry = 1 if (a[i] + b[i] + carry) > 1 else 0
    return result


a = bitarray(endian='big')
b = bitarray(endian='big')
a.frombytes(b'\x80\x30\xff\xff')
b.frombytes(b'\x80\x50\x00\x00')

print("{:>8} {:>32}".format("A:", a.to01()))
print("{:>8} {:>32} +".format("B:", b.to01()))
print("{:>8} {:>32}".format("", "-"*32))
out = add(a,b)
for i in range(32):
    print("\r{:>8} {:>32}".format("", out.to01()[32-1-i:]), end="")
    sleep(0.2)
del a, b, i

      A: 10000000001100001111111111111111
      B: 10000000010100000000000000000000 +
         --------------------------------
         00000000100000001111111111111111